In [ ]:
%load_ext autoreload

In [ ]:
from videoseg.youtube_vis import register_youtube_vis_from_dicts, get_dicts, get_thing_classes
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultPredictor, DefaultTrainer
import matplotlib.pyplot as plt
from pathlib import Path
import pycocotools.mask as mask_utils
import random
import cv2
import os
import matplotlib
import numpy as np

In [ ]:
path = "/mnt/datasets/public/youtube-vis/train/train_v2.json"
youtube_vis_train = MetadataCatalog.get("youtube-vis-train")
MetadataCatalog.get("youtube-vis-train").thing_classes = get_thing_classes(path)
dataset_dicts = get_dicts(path)

In [ ]:
dataset_dicts[0]["annotations"]

In [ ]:
print(MetadataCatalog.get("youtube-vis-train").thing_classes)

In [ ]:
for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=youtube_vis_train, scale=1.0)
    vis = visualizer.draw_dataset_dict(d)
    fig = plt.figure(figsize=(16, 16))
    ax1 = fig.add_subplot(1, 3, 1)
    ax1.imshow(vis.get_image())
    # get the mask from the previous frame
    tm1_np_mask = np.zeros((d["height"], d["width"]))
    for tm1_mask in d["tm1_mask"]:
        rle_mask = mask_utils.frPyObjects(tm1_mask, d["height"], d["width"])
        rle_mask = mask_utils.merge(rle_mask)
        tm1_np_mask = np.clip(tm1_np_mask + mask_utils.decode(rle_mask), 0, 1)
        
    ax2 = fig.add_subplot(1, 3, 2)
    ax2.imshow(tm1_np_mask, cmap="gray")
    ax3 = fig.add_subplot(1, 3, 3)
    img = cv2.imread(d["tm1_file_name"])
    ax3.imshow(np.array(img[:, :, ::-1]))

**Evaluation**